In [106]:
import glob
import os
import pandas as pd
import numpy as np
import sqlite3
import shutil
from random import randint

In [107]:
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

In [108]:
# load the tiles and their labels
TILE_BASE = '/Users/darylwilding-mcbride/Downloads/yolo-train-rt-3000-3600-denoised-30-apr'
TRAINING_SET_FILES_DIR = '{}/train'.format(TILE_BASE)
AUGMENTED_FILES_DIR = '{}/augmented'.format(TILE_BASE)
AUGMENTED_OVERLAY_FILES_DIR = '{}/overlay'.format(AUGMENTED_FILES_DIR)

In [109]:
# initialise the directories required for the data set creation
if os.path.exists(AUGMENTED_FILES_DIR):
    shutil.rmtree(AUGMENTED_FILES_DIR)
os.makedirs(AUGMENTED_FILES_DIR)
os.makedirs(AUGMENTED_OVERLAY_FILES_DIR)


In [110]:
# load the file names into a dataframe
filenames = []
for file in glob.glob("{}/*.png".format(TRAINING_SET_FILES_DIR)):
    filenames.append((os.path.basename(os.path.splitext(file)[0])))

In [111]:
filenames_df = pd.DataFrame(filenames, columns=['filename'])

In [112]:
filenames_df.head()

,filename
0,frame-32469-tile-20-mz-463-482
1,frame-30874-tile-71-mz-1392-1410
2,frame-29796-tile-56-mz-1119-1137
3,frame-31303-tile-77-mz-1501-1519
4,frame-29862-tile-25-mz-554-573


In [113]:
TILES_TO_BE_AUGMENTED_PERCENT = 0.01
AUGMENTATIONS_PER_TILE = 5
PIXELS = 910 # length of each tile edge in pixels
MAX_X = 300  # translation can occue up to this much in either direction
MAX_Y = 300

In [114]:
number_to_select = int(len(filenames_df) * TILES_TO_BE_AUGMENTED_PERCENT)

In [115]:
number_to_select

267

In [116]:
filenames_to_augment_df = filenames_df.sample(n=number_to_select)

In [117]:
filenames_to_augment_df.head()

,filename
16673,frame-29829-tile-58-mz-1155-1173
3393,frame-32513-tile-30-mz-645-664
3375,frame-32711-tile-56-mz-1119-1137
10838,frame-32315-tile-34-mz-718-736
25266,frame-32502-tile-62-mz-1228-1246


In [118]:
feature_label = ImageFont.truetype('/Library/Fonts/Arial.ttf', 10)
# feature_label = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf', 10)


In [120]:
for filename_idx in range(len(filenames_to_augment_df)):
    filename = filenames_to_augment_df.iloc[filename_idx].filename
    # load the tile
    img = Image.open('{}/{}.png'.format(TRAINING_SET_FILES_DIR, filename))
    # generate the augmented tiles
    for augmentation_idx in range(AUGMENTATIONS_PER_TILE):
        # apply the transformation
        x_offset = randint(-MAX_X, MAX_X+1)
        y_offset = randint(-MAX_Y, MAX_Y+1)
        # transformation matrix uses the inverse transformation
        # mapping (x,y) in the destination image to (ax + by + c, dx + ey + f) in the source image
        a = 1
        b = 0
        c = -x_offset # +ve is to the left, -ve is to the right
        d = 0
        e = 1
        f = -y_offset # +ve is up, -ve is down
        aug_img = img.transform(img.size, Image.AFFINE, (a, b, c, d, e, f))
        # now need to apply the same transformation to each label
        augment_label_l = []
        overlay_img = aug_img.copy() # copy the original so we can draw on it
        draw_context = ImageDraw.Draw(overlay_img)
        label_df = pd.read_csv('{}/{}.txt'.format(TRAINING_SET_FILES_DIR, filename), sep=' ', header=None, names=['class_id','x','y','w','h'])
        for label_idx in range(len(label_df)):
            class_id = label_df.iloc[label_idx].class_id
            label_x = label_df.iloc[label_idx].x
            label_y = label_df.iloc[label_idx].y
            label_width = label_df.iloc[label_idx].w
            label_height = label_df.iloc[label_idx].h
            # get the current pixel coordinates
            pixel_x = int(label_x * PIXELS)
            pixel_y = int(label_y * PIXELS)
            pixel_width = int(label_width * PIXELS)
            pixel_height = int(label_height * PIXELS)
            # calculate the new label centre x,y in pixel coordinates
            augment_pixel_x = pixel_x + x_offset # new centre in pixel coordinates
            augment_pixel_y = pixel_y + y_offset
            # calculate the new label centre x,y in label coordinates
            augment_label_x = augment_pixel_x / PIXELS # new centre in label coordinates
            augment_label_y = augment_pixel_x / PIXELS
            # label the object if its centre is still within the tile
            if ((augment_label_x >= 0) and (augment_label_x <= 1) and (augment_label_y >= 0) and (augment_label_y <= 1)):
                # add it to the list of new labels
                augment_label_l.append(("{} {:.6f} {:.6f} {:.6f} {:.6f}".format(class_id, augment_pixel_x, augment_pixel_y, label_width, label_height)))
                # calculate the overlay rectangle
                x0 = int(augment_pixel_x - (pixel_width / 2))
                x1 = int(augment_pixel_x + (pixel_width / 2))
                y0 = int(augment_pixel_y - (pixel_height / 2))
                y1 = int(augment_pixel_y + (pixel_height / 2))
                # draw the box annotation
                draw_context.rectangle(xy=[(x0, y0), (x1, y1)], fill=None, outline='red')
                draw_context.text((x0, y0-12), "feature class {}".format(class_id), font=feature_label, fill='red')
        # write out the augmented tile
        augmented_base_filename = '{}-aug-{}-{}'.format(filename, x_offset, y_offset)
        tile_filename = '{}/{}.png'.format(AUGMENTED_FILES_DIR, augmented_base_filename)
        aug_img.save(tile_filename)
        # save the label file
        label_filename = '{}/{}.txt'.format(AUGMENTED_FILES_DIR, augmented_base_filename)
        with open(label_filename, 'w') as f:
            for item in augment_label_l:
                f.write("%s\n" % item)
        # save the overlay image
        overlay_img.save('{}/{}.png'.format(AUGMENTED_OVERLAY_FILES_DIR, augmented_base_filename))
